1. Setup Environment 🔽

This cell downloads the initial setup.py script from your fork and then executes it, instructing the setup process to pull all subsequent project files from your repository.

In [ ]:
# @title 1. Setup Environment (Multi-Platform)

import os
from pathlib import Path

# --- Configuration ---
# Define the language, branch, and your target repository fork.
lang = 'en'
branch = 'main'
fork_repo = 'drf0rk/AnxietyLightning' # This should point to your repository.

# --- Script Execution Logic ---
# This block is now platform-agnostic. It lets setup.py handle platform detection.

# The setup.py script will be downloaded and run. It will automatically determine
# the correct HOME path for Colab, Kaggle, or Lightning AI.
scripts_dir = Path.home() / 'ANXETY' / 'scripts'
setup_script_path = scripts_dir / 'setup.py'

# Ensure the target directory for the script exists.
os.makedirs(scripts_dir, exist_ok=True)

# Download the setup.py script from your specified fork.
setup_url = f'https://raw.githubusercontent.com/{fork_repo}/{branch}/scripts/setup.py'
print(f"🔄 Downloading setup script from {fork_repo}...")
!curl -sLo {setup_script_path} {setup_url}
print("✅ Setup script downloaded.")

# Run the setup script. It will now automatically:
# 1. Detect the platform (Colab, Kaggle, Lightning AI).
# 2. Set the correct HOME directory.
# 3. Download all other necessary files from your specified fork.
%run {setup_script_path} --lang={lang} --branch={branch} --fork={fork_repo}

🔄 Changing directory from c:\Users\Greepo\Documents\GitHub\AnxietyLightning\Notebook to \teamspace\studios\this_studio


FileNotFoundError: [WinError 3] The system cannot find the path specified: '\\teamspace\\studios\\this_studio'

2. Widgets 🔽

This cell will load the interactive widgets. Since setup.py (executed in the previous cell) has already downloaded the patched widgets-en.py (or widgets-ru.py) to scripts_dir, this command will now use your modified version with the LoRA dropdown.

In [ ]:
# @title 2. Widgets (Corrected Path Logic)
from pathlib import Path
import sys

# Add the modules path to ensure json_utils can be imported
# The ANXETY folder itself is reliably placed relative to the initial home directory
anxety_path = Path.home() / 'ANXETY'
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js

# Read the correct scripts_dir path that was saved by setup.py
settings_path = anxety_path / 'settings.json'
scripts_dir = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))
lang = 'en'

print(f"✅ Running widget script from: {scripts_dir}")
%run {scripts_dir}/{lang}/widgets-{lang}.py

3. Downloading 🔽

This cell initiates the downloading process for models, VAEs, extensions, and other necessary components. It will use the patched downloading-en.py (or downloading-ru.py) from your fork, ensuring downloads go to your centralized model storage location.

In [ ]:
# @title 3. Downloading (Corrected Path Logic)
from pathlib import Path
import sys

# Add the modules path to ensure json_utils can be imported
anxety_path = Path.home() / 'ANXETY'
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js

# Read the correct scripts_dir path that was saved by setup.py
settings_path = anxety_path / 'settings.json'
scripts_dir = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))
lang = 'en'

print(f"✅ Running download script from: {scripts_dir}")
# The downloading script will now be found and executed correctly.
%run {scripts_dir}/{lang}/downloading-{lang}.py

📦 Processing download selections...
DEBUG: Selected Models: ['1. D5K6.0']
DEBUG: Selected VAEs: ['1. vae-ft-mse-840000-ema-pruned | 840000 | 840k SD1.5 VAE - vae-ft-mse-840k']
✨ Downloader Version: 2025.06.09.8_final_patch
📦 Processing download selections...
Starting downloads...
【#61abfb 1.7GiB/1.9GiB(90%) CN:16 DL:284MiB】                                                                                  
61abfb|OK  |   283MiB/s|/teamspace/studios/this_studio/sd_models_shared/Stable-diffusion/D5K6.0.safetensors
【#716d1f 169MiB/319MiB(53%) CN:16 DL:206MiB】                                                                                  
716d1f|OK  |   229MiB/s|/teamspace/studios/this_studio/sd_models_shared/vae/vaeFtMse840000EmaPruned_vaeFtMse840k.safetensors
【#bf6161 297MiB/297MiB(99%) CN:2 DL:205MiB】                                                                                   
bf6161|OK  |   204MiB/s|/teamspace/studios/this_studio/sd_models_shared/Lora/concept-female_masturbation-

4. Start 🔽

This cell launches the Stable Diffusion WebUI. It will execute the patched launch.py script from your fork, applying any platform-specific optimizations and arguments you've included.

In [ ]:
# @title 4. Start (Corrected Path Logic)
from pathlib import Path
import sys

# Add the modules path to ensure json_utils can be imported
anxety_path = Path.home() / 'ANXETY'
modules_path = anxety_path / 'modules'
if str(modules_path) not in sys.path:
    sys.path.insert(0, str(modules_path))

import json_utils as js

# Read the correct scripts_dir path that was saved by setup.py
settings_path = anxety_path / 'settings.json'
scripts_dir = Path(js.read(settings_path, 'ENVIRONMENT.scr_path'))

print(f"✅ Running launch script from: {scripts_dir}")
# Launch the WebUI. The -l flag provides more detailed logging for tunnels.
%run {scripts_dir}/launch.py -l

Utilities
5. Run Cleanup Utility 🔽

This cell runs the new Cleanup Utility GUI script, allowing you to manage your environment. This script is downloaded from your fork via the initial setup.

In [ ]:
# @title 5. Run Cleanup Utility

from pathlib import Path

# This cell is now platform-agnostic.
scripts_dir = Path.home() / 'ANXETY' / 'scripts'

# Run the cleanup utility GUI
%run {scripts_dir}/auto-cleaner.py

In [ ]:
from pathlib import Path
import shutil
import os
import sys

# --- Start of Directory Management ---
BASE_DIR = Path("/teamspace/studios/this_studio")
if os.getcwd() != str(BASE_DIR):
    print(f"🔄 Changing directory from {os.getcwd()} to {BASE_DIR}")
    os.chdir(BASE_DIR)
# --- End of Directory Management ---

# --- VERY IMPORTANT WARNING ---
# This cell will DELETE almost everything in your Lightning AI instance.
# It will only preserve this notebook file and 'main.py'.
# Ensure you understand what is being deleted before running.
# This operation is IRREVERSIBLE.

print("!!! DANGER: YOU ARE ABOUT TO DELETE ALMOST ALL FILES !!!")
print("!!! PLEASE READ CAREFULLY BEFORE PROCEEDING !!!")
print("\nThis cell will delete all folders and files in your current studio instance, EXCEPT:")
print(" - This notebook file (e.g., 'LightningAnxiety (1) (1) (2).ipynb')")
print(" - The 'main.py' file (if it exists in the root)")
print("\nTHIS WILL REQUIRE YOU TO RERUN THE ENTIRE NOTEBOOK FROM THE FIRST CELL FOR A FRESH START.")
print("If you have any custom files you wish to keep, MOVE THEM OUTSIDE THIS STUDIO INSTANCE NOW.")
print("Proceed only if you want a completely blank studio environment.")
print("\nType 'YES_DELETE_ALL' (case-sensitive) to confirm deletion and execute, then press Enter.")
print("Anything else will abort the operation.")

confirmation = input("Confirmation: ")

if confirmation.strip() == "YES_DELETE_ALL": # Case-sensitive comparison
    # Get the home/studio path
    # On Lightning AI, this is typically /teamspace/studios/this_studio
    HOME_PATH = Path.home()

    # Get the current notebook's filename
    # Use a more robust way to find the notebook file name, as __file__ is not always reliable in notebooks
    notebook_filename = "LightningAnxiety (1) (1) (2).ipynb" # Explicitly set your notebook filename here for reliability
    notebook_path = HOME_PATH / notebook_filename

    main_py_path = HOME_PATH / "main.py"

    # Define items to EXCLUDE from deletion
    EXCLUDE_LIST = [
        notebook_path,
        main_py_path
    ]

    print(f"\n--- Starting Comprehensive Deletion in {HOME_PATH} ---")
    deleted_count = 0
    skipped_count = 0

    for item in HOME_PATH.iterdir():
        # Convert item to absolute path for consistent comparison with EXCLUDE_LIST
        abs_item = item.resolve()
        if abs_item in EXCLUDE_LIST:
            print(f"ℹ️ Skipping protected item: {item.name}")
            skipped_count += 1
            continue

        print(f"🗑️ Attempting to delete: {item.name} ({item})...")
        try:
            if item.is_dir():
                shutil.rmtree(item)
            else:
                item.unlink() # Delete file
            print(f"✅ Successfully deleted: {item.name}")
            deleted_count += 1
        except Exception as e:
            print(f"❌ Error deleting {item.name} ({item}): {e}")

    print("\n--- Comprehensive Cleanup Process Complete ---")
    print(f"Summary: {deleted_count} items deleted, {skipped_count} items skipped (protected).")
    print("Please restart your runtime and run the notebook from the first cell for a fresh start.")
else:
    print("\nOperation aborted. No folders were deleted.\n")

# --- Ensure we are back in BASE_DIR at the end of the cell ---
if os.getcwd() != str(BASE_DIR):
    os.chdir(BASE_DIR)
# --- End of Directory Management ---
